# Collision Avoidance - Data Collection
# 避障 - 数据采集

If you ran through the basic motion notebook, hopefully you're enjoying how easy it can be to make your Jetbot move around! Thats very cool!  But what's even cooler, is making JetBot move around all by itself!  
如果你学会了通过基础移动的notbook轻松实现jetbot的行走，那就太了不起了。但更了不起的是让jetbot独自行走。  

This is a super hard task, that has many different approaches but the whole problem is usually broken down into easier sub-problems.  It could be argued that one of the most
important sub-problems to solve, is the problem of preventing the robot from entering dangerous situations!  We're calling this *collision avoidance*.  
这是一个超级难的任务，有许多不同的处理，但所有的问题通常会分解成更容易的子问题。而最重要的解决的问题是防止jetbot发生危险的情况，我们称之为避障。

In this set of notebooks, we're going to attempt to solve the problem using deep learning and a single, very versatile, sensor: the camera.  You'll see how with a neural network, camera, and the NVIDIA Jetson Nano, we can teach the robot a very useful behavior!
在这一套notebook，我们将会使用深度学习和一个非常通用的传感器：摄像头，来解决问题。你会学会如何使用神经网络，摄像头和NVIDIA Jetson Nano教会JetBot学会一个非常有用的行为！

The approach we take to avoiding collisions is to create a virtual "safety bubble" around the robot.  Within this safety bubble, the robot is able to spin in a circle without hitting any objects (or other dangerous situations like falling off a ledge).    
我们创建一个虚拟安全罩，在这个安全罩内机器人能够旋转一圈而不会碰撞到任何物体（或者其他情况，例如从桌面上掉落）。

Of course, the robot is limited by what's in it's field of vision, and we can't prevent objects from being placed behind the robot, etc.  But we can prevent the robot from entering these scenarios itself.  
当然，JetBot会受到视野的限制，我们无法防止物体被放置在JetBot后面等问题。但我们可以防止JetBot进入这些地方或场景。

The way we'll do this is super simple:  
我们的方式非常简单：

First, we'll manually place the robot in scenarios where it's "safety bubble" is violated, and label these scenarios ``blocked``.  We save a snapshot of what the robot sees along with this label.  
首先，我们会手动地把JetBot放置在违反安全罩的地方或场景中，把这些情景拍照并标记为``blocked``(阻塞)。

Second, we'll manually place the robot in scenarios where it's safe to move forward a bit, and label these scenarios ``free``.  Likewise, we save a snapshot along with this label.
其次，我们会手动地把JetBot放置放置在符合安全罩的地方或者场景，把这些情景拍照并标记为``free``(通畅)。

That's all that we'll do in this notebook; data collection.  Once we have lots of images and labels, we'll upload this data to a GPU enabled machine where we'll *train* a neural network to predict whether the robot's safety bubble is being violated based off of the image it sees.  We'll use this to implement a simple collision avoidance behavior in the end :)  
这就是我们在这个notebook所做的一切：数据采集。一旦我们有了大量的图像和标签，我们会把数据上传到支持GPU运算的主机上，*训练*一个神经网络，然后根据JetBot所看到的图像通过这个神经网络来判断安全罩是否被侵犯。最后，我们将使用这个神经网络来实现一个简单的避障行为！


> IMPORTANT NOTE:  When JetBot spins in place, it actually spins about the center between the two wheels, not the center of the robot chassis itself.  This is an important detail to remember when you're trying to estimate whether the robot's safety bubble is violated or not.  But don't worry, you don't have to be exact. If in doubt it's better to lean on the cautious side (a big safety bubble).  We want to make sure JetBot doesn't enter a scenario that it couldn't get out of by turning in place.  
>重要提示：当JetBot旋转的时候，它事件上是围绕着两个轮子之间的中心点旋转，而不是JetBot地盘的中心旋转。当你尝试预估JetBot旋转时安全罩是否被侵犯了的时候，这是一个重要的参考细节。但也不用太担心，不必太过于准确。如果有不放心的，最好就往更谨慎的方向做（例如虚拟一个更大的安全罩）。我们要确保JetBot不会进入一个无法转向而又无法离开的情况。

### Display live camera feed
### 实时显示摄像头

So let's get started.  First, let's initialize and display our camera like we did in the *teleoperation* notebook.  
那么，我们就开始了。首先，让我们像在*teleoperation* notebook中一样的初始化摄像头，并显示所看到的画面。

> Our neural network takes a 224x224 pixel image as input.  We'll set our camera to that size to minimize the filesize of our dataset (we've tested that it works for this task).  
> 我们的神经网络采用224x224像素的图像作为输入。我们将摄像头设置为该大小，以最小化文件大小，而最小化数据集。（我们已经通过测试此像素适用于此任务）
> In some scenarios it may be better to collect data in a larger image size and downscale to the desired size later.
> 在某些情况下，收集数据时最好用较大的图像尺寸，然后做处理的时候缩小到需要的大小。

In [1]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)

image = widgets.Image(format='jpeg', width=224, height=224)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Awesome, next let's create a few Awesome, next let's create a few directories where we'll store all our data.  We'll create a folder ``dataset`` that will contain two sub-folders ``free`` and ``blocked``, 
where we'll place the images for each scenario. where we'll store all our data.  We'll create a folder ``dataset`` that will contain two sub-folders ``free`` and ``blocked``, 
where we'll place the images for each scenario.  
太棒了，可以实时的看到摄像头拍摄到的画面。接下来让我们创建一些目录存储数据。我们将会建立一个叫``dataset``的文件夹，里面有两个子文件夹，分别是 ``free``和``blocked``，用于分类放置每一个场景。

In [2]:
import os

blocked_dir = 'dataset/blocked'
free_dir = 'dataset/free'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(free_dir)
    os.makedirs(blocked_dir)
except FileExistsError:
    print('Directories not created becasue they already exist')

If you refresh the Jupyter file browser on the left, you should now see those directories appear.  Next, let's create and display some buttons that we'll use to save snapshots
for each class label.  We'll also add some text boxes that will display how many images of each category that we've collected so far. This is useful because we want to make
sure we collect about as many ``free`` images as ``blocked`` images.  It also helps to know how many images we've collected overall.  

如果你刷新左侧的Jupyter文件浏览器，你现在应该会见到这些目录。接下来，我们将创建一些按钮用来保存不同标签的快照。我们还将创建一些文本框，用于显示到目前位置我们每个标签收集到的图像数量。这很重要，因为我们要确保采集到的``free``图像要和``blocked``图像一样多。还有助于我们了解整体收集了多少图像。

In [3]:
button_layout = widgets.Layout(width='128px', height='64px')
free_button = widgets.Button(description='add free', button_style='success', layout=button_layout)
blocked_button = widgets.Button(description='add blocked', button_style='danger', layout=button_layout)
free_count = widgets.IntText(layout=button_layout, value=len(os.listdir(free_dir)))
blocked_count = widgets.IntText(layout=button_layout, value=len(os.listdir(blocked_dir)))

display(widgets.HBox([free_count, free_button]))
display(widgets.HBox([blocked_count, blocked_button]))

Right now, these buttons wont do anything.  We have to attach functions to save images for each category to the buttons' ``on_click`` event.  We'll save the value
of the ``Image`` widget (rather than the camera), because it's already in compressed JPEG format!  
到此为止，那些按钮是不会做任何事的。我们需要把保存图像的函数附加到每一个按钮的``on_click`` 事件中。我们会通过``Image``部件，保存这些数值，因为这已经是经过压缩处理的JEPEG格式图像。

To make sure we don't repeat any file names (even across different machines!) we'll use the ``uuid`` package in python, which defines the ``uuid1`` method to generate
a unique identifier.  This unique identifier is generated from information like the current time and the machine address.  
为了确保我们不重复的任何文件名（甚至跨越不同的机器！）我们将在python中使用``uuid``  package，它的作用是可以定义``uuid``方法，生成唯一标识符。该标识符是根据当前机器地址和时间生成的。

In [4]:
from uuid import uuid1

def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)

def save_free():
    global free_dir, free_count
    save_snapshot(free_dir)
    free_count.value = len(os.listdir(free_dir))
    
def save_blocked():
    global blocked_dir, blocked_count
    save_snapshot(blocked_dir)
    blocked_count.value = len(os.listdir(blocked_dir))
    
# attach the callbacks, we use a 'lambda' function to ignore the
# parameter that the on_click event would provide to our function
# because we don't need it.
free_button.on_click(lambda x: save_free())
blocked_button.on_click(lambda x: save_blocked())

Great! Now the buttons above should save images to the ``free`` and ``blocked`` directories.  You can use the Jupyter Lab file browser to view these files!  
太棒了，现在上面的按钮应该可以把图片保存在``free`` 或者 ``blocked`` 文件夹里。你可以使用Jupyter Labde 的文件浏览器去查看这些文件。

Now go ahead and collect some data  
现在继续收集一些数据

1. Place the robot in a scenario where it's blocked and press ``add blocked``  
1，请把JetBot放在阻挡的情况下，并按下``add blocked``按钮
2. Place the robot in a scenario where it's free and press ``add free``  
2，请把JetBot放在通畅的情况下，并按下``add free``按钮
3. Repeat 1, 2  
3.重复1，2

> REMINDER: You can move the widgets to new windows by right clicking the cell and clicking ``Create New View for Output``.  Or, you can just re-display them
> together as we will below
> 提示：你可以把那些按钮部件输出在新的窗口，这样方便你的操作。我们也将执行下面的代码把它们显示在一起。

Here are some tips for labeling data
以下是一些数据标记的提示

1. Try different orientations  
1. 尝试不同方向  
2. Try different lighting  
2. 尝试不同的照明  
3. Try varied object / collision types; walls, ledges, objects  
3. 尝试不同的对象/碰撞类型：例如墙壁，物体等  
4. Try different textured floors / objects;  patterned, smooth, glass, etc.  
4. 尝试不同纹理的平面/物体：例如图案，不同光滑度，玻璃等

Ultimately, the more data we have of scenarios the robot will encounter in the real world, the better our collision avoidance behavior will be.  It's important
to get *varied* data (as described by the above tips) and not just a lot of data, but you'll probably need at least 100 images of each class (that's not a science, just a helpful tip here).  But don't worry, it goes pretty fast once you get going :)  
最终，我们拥有JetBot在现实世界中越到的场景数据越多，我们的防撞行为就越好。得到各种各样的数据很重要，而不仅仅是大量的数据。可能每个分类都需要100个图像（这不一定是一个科学的做法，仅仅是一个有用的提示）。这么多数据其实不用担心，当你开始收集的时候，就会很快完成。

In [ ]:
display(image)
display(widgets.HBox([free_count, free_button]))
display(widgets.HBox([blocked_count, blocked_button]))

## Next
## 下一步

Once you've collected enough data, we'll need to copy that data to our GPU desktop or cloud machine for training.  First, we can call the following *terminal* command to compress
our dataset folder into a single *zip* file.  
当你收集足够的数据的时候，我们需要把这些数据复制到我们的GPU平台上进行训练。首先，我们可以调用*terminal*（命令行模式又或者叫终端）命令，进行数据打包压缩成一个*.zip文件。


> The ! prefix indicates that we want to run the cell as a *shell* (or *terminal*) command.  
> ! 表示我们要将使用shell命令运行

> The -r flag in the zip command below indicates *recursive* so that we include all nested files, the -q flag indicates *quiet* so that the zip command doesn't print any output  
> -r 表示包含所有包含子文件夹文件。-q 表示zip命令不输出任何信息

In [ ]:
!zip -r -q dataset.zip dataset

You should see a file named ``dataset.zip`` in the Jupyter Lab file browser.  You should download the zip file using the Jupyter Lab file browser by right clicking and selecting ``Download``.  
您应该在Jupyter Lab文件浏览器中看到名为``dataset.zip``的文件。你可以右键点击改文件进行下载。 

Next, we'll need to upload this data to our GPU desktop or cloud machine (we refer to this as the *host*) to train the collision avoidance neural network.  We'll assume that you've set up your training
machine as described in the JetBot WiKi.  If you have, you can navigate to ``http://<host_ip_address>:8888`` to open up the Jupyter Lab environment running on the host.  The notebook you'll need to open there is called ``collision_avoidance/train_model.ipynb``.  
接下来，我们需要把这些数据上传到我们的GPU平台或者云计算机来训练我们的防撞神经网络。Jetson Nano是支持GPU的，所有接下来，我们直接在这里打开``train_model.ipynb``这个模型训练的notebook。
  
So head on over to your training machine and follow the instructions there!  Once your model is trained, we'll return to the robot Jupyter Lab enivornment to use the model for a live demo!  
所以，请继续前往训练操作，一步一步地按说明运行，这将会非常简单。
